In [1]:
# Importamos librerías que necesitaremos
import ftplib # Para la conexion con el servidor y descargar el archivo
import xml.etree.ElementTree as et # Para trabajarel fichero .xml
import pandas as pd
import numpy as np

#### SE PIDE:
##### A partir del fichero OptaF24.xml almacenado en un servidor FTP, que contiene los eventos de un partido concreto, crear un dataframe que contenga todos los pases de dicho partido. Este dataframe deberá contener los siguientes campos (columnas):

##### 1) Descarga el archivo OptaF24.xml creando una conexión al servidor FTP
- Servidor: f25-preview.runhosting.com
- Usuario: 3185129_EOI
- Contraseña: FTP_EOI1920


In [2]:
# Creamos la conexión con el servidor FTP
ftp = ftplib.FTP('f25-preview.runhosting.com','3185129_EOI','FTP_EOI1920')
# Descargamos el archivo OptaF24.xml 
filename = 'OptaF24.xml'
ftp.retrbinary("RETR " + filename, open(filename,'wb').write)

'226 Transfer complete'

##### Antes de nada vamos inspeccionar el xml


In [3]:
# Veamos las primeras 20 lineas
with open('./OptaF24.xml') as f:
    for x in range(20):
        print(f.readline().strip())

<?xml version="1.0" encoding="UTF-8"?>
<!-- Copyright 2001-2012 Opta Sportsdata Ltd. All rights reserved. -->

<!-- PRODUCTION HEADER
produced on:        valde-jobq-a03.nexus.opta.net
production time:    20120911T092033,103Z
production module:  Opta::Feed::XML::Soccer::F24
-->
<Games timestamp="2012-09-11T10:20:32">
<Game id="360481" away_team_id="43" away_team_name="Manchester City" competition_id="8" competition_name="English Barclays Premier League" game_date="2011-08-21T16:00:00" home_team_id="30" home_team_name="Bolton Wanderers" matchday="2" period_1_start="2011-08-21T16:00:38" period_2_start="2011-08-21T17:03:47" season_id="2011" season_name="Season 2011/2012">
<Event id="301038339" event_id="1" type_id="34" period_id="16" min="0" sec="0" team_id="43" outcome="1" x="0.0" y="0.0" timestamp="2011-08-21T15:23:06.696" last_modified="2011-08-21T15:54:56">
<Q id="2028397186" qualifier_id="130" value="4" />
<Q id="1518776786" qualifier_id="227" value="0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [4]:
tree = et.parse('./OptaF24.xml')
raiz = tree.getroot()
print ('Raiz: ', raiz.tag)

Raiz:  Games


In [6]:
for elem in tree.iter():
    print (elem)

<Element 'Games' at 0x0000026FDAE9A8B0>
<Element 'Game' at 0x0000026FDAE9AB30>
<Element 'Event' at 0x0000026FDAE9AC20>
<Element 'Q' at 0x0000026FDAE9AC70>
<Element 'Q' at 0x0000026FDAE9ACC0>
<Element 'Q' at 0x0000026FDAE9AD10>
<Element 'Q' at 0x0000026FDAE9AD60>
<Element 'Q' at 0x0000026FDAE9ADB0>
<Element 'Q' at 0x0000026FDAE9AE00>
<Element 'Q' at 0x0000026FDAE9AE50>
<Element 'Event' at 0x0000026FDAE9AF40>
<Element 'Q' at 0x0000026FDAE9AF90>
<Element 'Q' at 0x0000026FDAEA7040>
<Element 'Q' at 0x0000026FDAEA7090>
<Element 'Q' at 0x0000026FDAEA70E0>
<Element 'Q' at 0x0000026FDAEA7130>
<Element 'Q' at 0x0000026FDAEA7180>
<Element 'Q' at 0x0000026FDAEA71D0>
<Element 'Event' at 0x0000026FDAEA72C0>
<Element 'Q' at 0x0000026FDAEA7310>
<Element 'Event' at 0x0000026FDAEA7400>
<Element 'Q' at 0x0000026FDAEA7450>
<Element 'Event' at 0x0000026FDAEA7540>
<Element 'Q' at 0x0000026FDAEA7590>
<Element 'Q' at 0x0000026FDAEA75E0>
<Element 'Q' at 0x0000026FDAEA7630>
<Element 'Q' at 0x0000026FDAEA7680>
<

In [12]:
# Para conocer todos los elementos únicos del árbol y cuántos hay de cada uno deellos
elementos = np.array([])
for elem in raiz.iter():
    elementos = np.append(elementos, elem.tag)

for elem in np.unique(elementos):
      print (elem, 'aparece:', np.count_nonzero(elementos==elem))

Event aparece: 1673
Game aparece: 1
Games aparece: 1
Q aparece: 6601


Por tanto, solo tenemos un único partido, con 1.673 eventos y dentro de estos eventos tenemos en total 6.601 cualificadores

##### 2) Crea una lista inicialmente vacía para cada una de las columnas del dataframe

In [27]:
# Creamos las listas vacias que tendrán el contenido de cada columna del dataframe
team = []
half = []
minute = []
second = []
x_origin = []
y_origin = []
x_destination = []
y_destination = []
outcome = []

##### 4) Itera cada evento y filtra aquellos que correspondan a pases (atributo type_id="1"), añadiendo a cada lista del paso anterior los atributos correspondientes de cada evento. Para obtener el valor de un atributo puedes utilizar la función attrib.get()


In [28]:
# Creamos una variable que nos ayude a saber si el evento anterior fue un pase
# Para que cuando entremos solo en las Q que realmente vienen precedidas de un pase
# Esto se hace tras ver que hay 956 Q con qualifier_id == 140|141, mientras que hay
# 946 pases (type_id == '1')
val = 0
for elem in raiz.iter():
    if (elem.tag == 'Event') & (elem.attrib.get('type_id') == '1'):
        val = 1 
        # Añadimos el equipo dependiendo del team_id
        if elem.attrib.get('team_id')=='43':
            team.append('Manchester City')
        elif elem.attrib.get('team_id')=='30':
            team.append('Bolton Wanderers')
        # Añadimos el resto de valores a las listas
        half.append(elem.attrib.get('period_id'))
        minute.append(elem.attrib.get('min'))
        second.append(elem.attrib.get('sec'))
        x_origin.append(elem.attrib.get('x'))
        y_origin.append(elem.attrib.get('y'))
        outcome.append(elem.attrib.get('outcome'))
    
    elif (val == 1) & (elem.tag == 'Q') & (elem.attrib.get('qualifier_id') == '140'):    
        x_destination.append(elem.attrib.get('value'))
    elif (val == 1) & (elem.tag == 'Q') & (elem.attrib.get('qualifier_id') == '141'):    
        y_destination.append(elem.attrib.get('value'))
        
    elif (val == 1) & (elem.tag == 'Q'):
        val = 1
    
    else:
        val = 0

Nota: Cuando hice esta tarea pensé que me había complicado mucho al hacer el bucle y decidí volver a repasar este ejercicio antes de subir la tarea. Creo que efectivamente me compliqué y a continuación lo hago de una forma algo más fácil.

In [31]:
# Volvemos a crear las listas
team = []
half = []
minute = []
second = []
x_origin = []
y_origin = []
x_destination = []
y_destination = []
outcome = []


for elem in raiz.iter():
    if elem.attrib.get('type_id')=='1':
        equipo = elem.attrib.get('team_id')
        if equipo == '43':
            team.append('Manchester City')
        elif equipo == '30':
            team.append('Bolton Wanderers')
        half.append(elem.attrib.get('period_id'))
        minute.append(elem.attrib.get('min'))
        second.append(elem.attrib.get('sec'))
        x_origin.append(elem.attrib.get('x'))
        y_origin.append(elem.attrib.get('y'))
        outcome.append(elem.attrib.get('outcome'))
        for q in elem:
            if q.attrib.get('qualifier_id') == '140':
                x_destination.append(q.attrib.get('value'))
            elif q.attrib.get('qualifier_id') == '141':
                y_destination.append(q.attrib.get('value'))

In [32]:
# Creamos el dataframe introduciendo las listas en un diccionario
data = {'team': team,
        'half': half,
        'minute': minute,
        'second': second,
        'x_origin': x_origin,
        'y_origin': y_origin,
        'x_destination': x_destination,
        'y_destination': y_destination,
        'outcome': outcome}

df = pd.DataFrame(data, columns=data.keys())
df.head()

,team,half,minute,second,x_origin,y_origin,x_destination,y_destination,outcome
0,Manchester City,1,0,1,50.1,50.0,52.4,49.1,1
1,Manchester City,1,0,2,48.2,49.1,29.0,76.5,1
2,Manchester City,1,0,19,27.8,100.0,49.3,93.4,0
3,Bolton Wanderers,1,0,23,50.9,20.0,63.6,30.1,1
4,Manchester City,1,0,30,29.3,73.0,30.2,55.6,1


#### OPCIONAL: 
##### Obtén los pases de más de 20 metros de avance en la coordenada x. ¿Qué equipo ha efectuado más veces estos pases?

- Consideraciones del alumno:
    - El campo del Bolton Wanderers mide 100.6 metros de largo
    - Solo se consideran pases éxitosos
    - Solo se consideran pases que vayan en direccion del ataque (avance)

In [35]:
# Primero hacemos la resta entre la coordenada de destino y la de origen
# Si el resultado es positivo será un avance
df['dif_x'] = df['x_destination'].astype(float) - df['x_origin'].astype(float)
# Lo pasamos a metros
df['dif_x_meters'] = df['dif_x']*100.6/100 

In [36]:
# Realizamos el calculo con las condiciones indicadas
temp = df.loc[(df.dif_x_meters>20) & (df.outcome=='1')]

temp = temp[['team','outcome']].groupby('team', as_index=False).count().sort_values('outcome', ascending=False)

print ('El ', temp.team.values[0], ' hizo ',  temp.outcome.values[0], ' pases de más de 20 metros')
print ('El ', temp.team.values[1], ' hizo ',  temp.outcome.values[1], ' pases de más de 20 metros')

El  Bolton Wanderers  hizo  36  pases de más de 20 metros
El  Manchester City  hizo  32  pases de más de 20 metros
